In [1]:
import pandas as pd
import sys
from pathlib import Path
base_dir = str(Path(Path.cwd()).resolve().parent.parent)
print(base_dir)
sys.path.append(base_dir)

B:\data_science_projects\erdos_institute\aware-nlp


In [2]:
from src.retriever import CustomRetriever, QdrantRetriever, ChromaRetriever, BaseRetriever

c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sample_file = '../../data/best_buy/questions_statements_labels.csv'
df = pd.read_csv(sample_file).drop(columns=['Unnamed: 0'])
df

,statement,question,label_sum,average_label,consensus,reddit_id,aware_post_type,aware_created_ts,reddit_link_id,reddit_parent_id,reddit_permalink,reddit_subreddit
0,![gif](giphy|znRstrOYuirrW),What do Best Buy employees think of the company?,0.0,0.000000,0,ju59a0l,comment,2023-07-30T21:11:35,t3_15e1vvl,t3_15e1vvl,/r/BestBuyWorkers/comments/15e1vvl/customer_po...,BestBuyWorkers
1,#shockedpikachuface,What do Best Buy employees think of the company?,0.0,0.000000,0,hhgbgps,comment,2021-10-21T00:06:32,t3_qafhhx,t1_hh3zf24,/r/BestBuyWorkers/comments/qafhhx/we_can_impro...,BestBuyWorkers
2,12 hr shift here too. Normal pay man,What are the most common reasons for employees...,2.0,0.285714,1,iwshpu5,comment,2022-11-17T19:35:11,t3_yy085z,t3_yy085z,/r/BestBuyWorkers/comments/yy085z/black_friday...,BestBuyWorkers
3,12-day Application Review; What is the usual d...,What are the most common reasons for employees...,0.0,0.000000,0,klk0z1,submission,2020-12-28T00:22:54,NaN,NaN,/r/BestBuyWorkers/comments/klk0z1/12day_applic...,BestBuyWorkers
4,Absolutely. I had a talk with a leader last we...,Do employees feel understaffed?,7.0,1.000000,1,ibg921p,comment,2022-06-07T00:11:03,t3_v5thte,t1_ibd38u3,/r/BestBuyWorkers/comments/v5thte/what_are_som...,BestBuyWorkers
...,...,...,...,...,...,...,...,...,...,...,...,...
85,done company wide every March not on your work...,What do Best Buy employees think of the company?,0.0,0.000000,0,k01yx5z,comment,2023-09-10T21:51:15,t3_16fik5c,t3_16fik5c,/r/BestBuyWorkers/comments/16fik5c/so_are_annu...,BestBuyWorkers
86,i take only cash tips from anyone thats not a ...,Do employees feel understaffed?,0.0,0.000000,0,ki7m0f4,comment,2024-01-16T19:53:41,t3_198a0ow,t3_198a0ow,/r/BestBuyWorkers/comments/198a0ow/customer_ti...,BestBuyWorkers
87,"i've been here for 8 years, work in the highes...",What are the most common reasons for employees...,7.0,1.000000,1,jag8nz0,comment,2023-03-01T00:13:02,t3_11dxaol,t3_11dxaol,/r/BestBuyWorkers/comments/11dxaol/new_to_this...,BestBuyWorkers
88,nah,What do Best Buy employees think of the company?,0.0,0.000000,0,ixd7txp,comment,2022-11-22T10:51:56,t3_z1mzn6,t3_z1mzn6,/r/BestBuyWorkers/comments/z1mzn6/bf_walk_out/...,BestBuyWorkers


In [4]:
from langchain_community.document_loaders import DataFrameLoader
import numpy as np

def parse_dataset(dataset_df: pd.DataFrame, question: str):
    df = dataset_df[dataset_df.question==question].replace({np.nan: ''})
    loader = DataFrameLoader(df, page_content_column='statement')
    return loader.load()

dataset_df = df.copy().drop(columns = ['label_sum', 'average_label', 'consensus'])
questions = dataset_df.question.unique()
dataset_dict = {f'question_{i}':parse_dataset(dataset_df, q) for i, q in enumerate(questions)}

In [5]:
docs = dataset_dict['question_0']

### Custom Retriever Example

In [8]:

encoder_name = 'multi-qa-mpnet-base-dot-v1'
retr = CustomRetriever(docs, encoder_name, 'euclidian')

In [9]:
retr.load_documents()

In [10]:
output = retr.retrieve(docs[0].metadata['question'])
output

[{'document': Document(page_content='Life as a Best Buy worker 💀', metadata={'question': 'What do Best Buy employees think of the company?', 'reddit_id': '134bzme', 'aware_post_type': 'submission', 'aware_created_ts': '2023-04-30T23:58:58', 'reddit_link_id': '', 'reddit_parent_id': '', 'reddit_permalink': '/r/BestBuyWorkers/comments/134bzme/life_as_a_best_buy_worker/', 'reddit_subreddit': 'BestBuyWorkers'}),
  'score': 4.1177473},
 {'document': Document(page_content='Don’t listen to this guy, I work there and the team environment is outstanding everyone stands around talking to each other and let’s the antisocial people ring up the customers. You’ll enjoy Best Buy as long as you aren’t antisocial and you actually enjoy technology', metadata={'question': 'What do Best Buy employees think of the company?', 'reddit_id': 'iw9vtz5', 'aware_post_type': 'comment', 'aware_created_ts': '2022-11-13T20:26:31', 'reddit_link_id': 't3_yuacvb', 'reddit_parent_id': 't1_iw8gy71', 'reddit_permalink': '/

#### Qdrant Implementation

In [11]:
retr_qdrant = QdrantRetriever(docs, encoder_name, 'euclidian')
retr_qdrant.load_documents()

In [12]:
output = retr_qdrant.retrieve(docs[0].metadata['question'])
output

[{'document': Document(page_content='Life as a Best Buy worker 💀', metadata={'question': 'What do Best Buy employees think of the company?', 'reddit_id': '134bzme', 'aware_post_type': 'submission', 'aware_created_ts': '2023-04-30T23:58:58', 'reddit_link_id': '', 'reddit_parent_id': '', 'reddit_permalink': '/r/BestBuyWorkers/comments/134bzme/life_as_a_best_buy_worker/', 'reddit_subreddit': 'BestBuyWorkers'}),
  'score': 4.117746998627767},
 {'document': Document(page_content='Don’t listen to this guy, I work there and the team environment is outstanding everyone stands around talking to each other and let’s the antisocial people ring up the customers. You’ll enjoy Best Buy as long as you aren’t antisocial and you actually enjoy technology', metadata={'question': 'What do Best Buy employees think of the company?', 'reddit_id': 'iw9vtz5', 'aware_post_type': 'comment', 'aware_created_ts': '2022-11-13T20:26:31', 'reddit_link_id': 't3_yuacvb', 'reddit_parent_id': 't1_iw8gy71', 'reddit_permal

#### Chroma

In [ ]:
!pip install chromadb

In [13]:
retr_chroma = ChromaRetriever(docs, encoder_name, 'euclidian')

In [14]:
retr_chroma.load_documents()

In [15]:
output = retr_qdrant.retrieve(docs[0].metadata['question'])
output

[{'document': Document(page_content='Life as a Best Buy worker 💀', metadata={'question': 'What do Best Buy employees think of the company?', 'reddit_id': '134bzme', 'aware_post_type': 'submission', 'aware_created_ts': '2023-04-30T23:58:58', 'reddit_link_id': '', 'reddit_parent_id': '', 'reddit_permalink': '/r/BestBuyWorkers/comments/134bzme/life_as_a_best_buy_worker/', 'reddit_subreddit': 'BestBuyWorkers'}),
  'score': 4.117746998627767},
 {'document': Document(page_content='Don’t listen to this guy, I work there and the team environment is outstanding everyone stands around talking to each other and let’s the antisocial people ring up the customers. You’ll enjoy Best Buy as long as you aren’t antisocial and you actually enjoy technology', metadata={'question': 'What do Best Buy employees think of the company?', 'reddit_id': 'iw9vtz5', 'aware_post_type': 'comment', 'aware_created_ts': '2022-11-13T20:26:31', 'reddit_link_id': 't3_yuacvb', 'reddit_parent_id': 't1_iw8gy71', 'reddit_permal

## RetrieverEvaluator

In [23]:
from src.retriever import BaseRetriever
from typing import Literal, List, Dict, Any
from langchain_core.documents import Document
from sklearn.metrics import recall_score, f1_score, precision_score
from langchain_community.document_loaders import DataFrameLoader
import numpy as np

class RetrieverEvaluator:
    def __init__(
            self, 
            retriever_name: Literal['choma', 'qdrant', 'custom'], 
            encoder_name: str,
            similarity_metric: str,
            sample_df: pd.DataFrame, 
            retrieved_doc_size: int = 10
        ):
        self.retriever_name = retriever_name
        self.sample_df = sample_df
        self.similarity_name = similarity_metric
        self.encoder_name = encoder_name
        self.n_retrieved = retrieved_doc_size
        self.questions = sample_df.question.unique()
        self.dataset_dict = self.parse_dataset()

    def get_retriever(self, docs: List[Document]) -> BaseRetriever:
        match self.retriever_name:
            case 'chroma':
                return ChromaRetriever(docs, self.encoder_name, self.similarity_name)
            case 'qdrant':
                return QdrantRetriever(docs, self.encoder_name, self.similarity_name)
            case 'custom':
                return CustomRetriever(docs, self.encoder_name, self.similarity_name)
            case _:
                raise ValueError(f"Retriever {self.retriever_name} not supported")

    def _parse_dataset_one_question(self, question: str):
        df = self.sample_df.drop(columns = ['label_sum', 'average_label', 'consensus']).copy()
        df = df[dataset_df.question==question].fillna('')
        loader = DataFrameLoader(df, page_content_column='statement')
        return loader.load()
        
    def parse_dataset(self):
        return {
            q:self._parse_dataset_one_question(q) 
            for q in self.questions
        }

    def retrieve_one_question(self, question: str):
        retriever = self.get_retriever(self.dataset_dict[question])
        retriever.load_documents()
        retrieved_docs = retriever.retrieve(question, n = self.n_retrieved)
        return retrieved_docs
    
    def _parse_retriever_element(self, elem: Dict[str, Any]):
        return {'statement':elem['document'].page_content, 'score':elem['score']} | elem['document'].metadata

    def _construct_evaluation_df(self, question: str, output: List[Dict[str, Any]]) -> pd.DataFrame:
        df_pred = pd.DataFrame([self._parse_retriever_element(elem) for elem in output])
        df_pred['retrieved'] = 1
        return df[df.question==question].merge(df_pred, how='outer').fillna({'retrieved':0, 'consensus':0})

    def evaluate_one_question(self, question: str):
        retrieved_docs = self.retrieve_one_question(question)
        eval_df = self._construct_evaluation_df(question, retrieved_docs)

        f1 = f1_score(eval_df['consensus'], eval_df['retrieved'])
        recall = recall_score(eval_df['consensus'], eval_df['retrieved'])
        precision = precision_score(eval_df['consensus'], eval_df['retrieved'])
        
        return {
            'question':question, 
            'retriever':self.retriever_name, 
            'encoder':self.encoder_name,
            'similarity':self.similarity_name,
            'f1':f1, 
            'recall':recall, 
            'precision':precision
        }

    def evaluate(self):
        return pd.DataFrame([self.evaluate_one_question(q) for q in self.questions])
            

In [24]:

encoder_metric_dict = {
    'multi-qa-mpnet-base-dot-v1': ['dot'],
    'multi-qa-mpnet-base-cos-v1': ['euclidian', 'cosine', 'dot'],
    'all-mpnet-base-v2': ['euclidian', 'cosine', 'dot'],
    'gtr-t5-xl': ['euclidian', 'cosine', 'dot'],
    'gtr-t5-xxl': ['euclidian', 'cosine', 'dot'],
    'gtr-t5-large': ['euclidian', 'cosine', 'dot'],
    'all-mpnet-base-v1': ['euclidian', 'cosine', 'dot'],
    'multi-qa-distilbert-cos-v1': ['euclidean', 'cosine', 'dot'],
    'multi-qa-MiniLM-L6-cos-v1': ['euclidean', 'cosine', 'dot'],
}
n_docs = 10

encoders = [
    'multi-qa-mpnet-base-dot-v1',
    'multi-qa-mpnet-base-cos-v1',
    'all-mpnet-base-v2'
]

output_df = pd.DataFrame()

for encoder_name in encoders:
    for similarity_metric in encoder_metric_dict[encoder_name]:
        retr_evaluator = RetrieverEvaluator('custom', encoder_name, similarity_metric, df, retrieved_doc_size=n_docs)
        output = retr_evaluator.evaluate()
        print(output)
        output_df = pd.concat([output_df, output])


                                            question retriever  \
0   What do Best Buy employees think of the company?    custom   
1  What are the most common reasons for employees...    custom   
2                    Do employees feel understaffed?    custom   

                      encoder similarity   f1  recall  precision  
0  multi-qa-mpnet-base-dot-v1        dot  0.7     0.7        0.7  
1  multi-qa-mpnet-base-dot-v1        dot  0.5     0.5        0.5  
2  multi-qa-mpnet-base-dot-v1        dot  0.6     0.6        0.6  
                                            question retriever  \
0   What do Best Buy employees think of the company?    custom   
1  What are the most common reasons for employees...    custom   
2                    Do employees feel understaffed?    custom   

                      encoder similarity   f1  recall  precision  
0  multi-qa-mpnet-base-cos-v1  euclidian  0.7     0.7        0.7  
1  multi-qa-mpnet-base-cos-v1  euclidian  0.4     0.4        0.4  
2

In [25]:
output_df.pivot_table(index=['question', 'retriever', 'encoder','similarity'], values=['f1', 'recall', 'precision'])

f1  \
question                                           retriever encoder                    similarity        
Do employees feel understaffed?                    custom    all-mpnet-base-v2          cosine      0.6   
                                                                                        dot         0.6   
                                                                                        euclidian   0.6   
                                                             multi-qa-mpnet-base-cos-v1 cosine      0.6   
                                                                                        dot         0.6   
                                                                                        euclidian   0.6   
                                                             multi-qa-mpnet-base-dot-v1 dot         0.6   
What are the most common reasons for employees ... custom    all-mpnet-base-v2          cosine      0.5   
                                                                                        dot         0.5   
                                                                                        euclidian   0.5   
                                                             multi-qa-mpnet-base-cos-v1 cosine      0.4   
                                                                                        dot         0.4   
                                                                                        euclidian   0.4   
                                                             multi-qa-mpnet-base-dot-v1 dot         0.5   
What do Best Buy employees think of the company?   custom    all-mpnet-base-v2          cosine      0.7   
                                                                                        dot         0.7   
                                                                                        euclidian   0.7   
                                                             multi-qa-mpnet-base-cos-v1 cosine      0.7   
                                                                                        dot         0.7   
                                                                                        euclidian   0.7   
                                                             multi-qa-mpnet-base-dot-v1 dot         0.7   

                                                                                                    precision  \
question                                           retriever encoder                    similarity              
Do employees feel understaffed?                    custom    all-mpnet-base-v2          cosine            0.6   
                                                                                        dot               0.6   
                                                                                        euclidian         0.6   
                                                             multi-qa-mpnet-base-cos-v1 cosine            0.6   
                                                                                        dot               0.6   
                                                                                        euclidian         0.6   
                                                             multi-qa-mpnet-base-dot-v1 dot               0.6   
What are the most common reasons for employees ... custom    all-mpnet-base-v2          cosine            0.5   
                                                                                        dot               0.5   
                                                                                        euclidian         0.5   
                                                             multi-qa-mpnet-base-cos-v1 cosine            0.4   
                                                                                        dot               0.4   
                                                         

In [27]:
output_df.pivot_table(index=['retriever', 'encoder','similarity'], values=['f1', 'recall', 'precision'])[['f1']]

f1
retriever encoder                    similarity          
custom    all-mpnet-base-v2          cosine      0.600000
                                     dot         0.600000
                                     euclidian   0.600000
          multi-qa-mpnet-base-cos-v1 cosine      0.566667
                                     dot         0.566667
                                     euclidian   0.566667
          multi-qa-mpnet-base-dot-v1 dot         0.600000